In [35]:
import matplotlib
from matplotlib import pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from scipy import stats
import collections as col
from IPython.display import IFrame
from geopy import distance
import statistics
import math
import dgl

# import geopy
# from geopy import distance
from geolib import geohash as gh
import json
nodes_enc = pd.read_csv('nodes_encoded_11_21', sep='\t')
er = pd.read_csv('edge_enc_11_21', sep='\t')
er

,type,weight,source,target
0,0,10.640525,1300,742
1,0,15.202562,1300,291
2,0,4.457279,1300,1245
3,0,19.127800,1300,412
4,0,16.647304,1300,3167
...,...,...,...,...
802854,0,19.403629,469,1464
802855,0,12.932615,2622,814
802856,0,7.594553,2622,717
802857,0,17.301979,2523,814


In [39]:

from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn.functional as F
lb_make = LabelEncoder()
# Load and encode node features

er = pd.read_csv('edges_remove_11_20', sep='\t')
nodes_cpy = pd.read_csv('nodes_cpy',sep='\t')

src = np.asarray(er['source'].to_numpy()).astype(int)
dst = np.asarray(er['target'].to_numpy()).astype(int)

# # # Create a DGL graph from a pair of numpy arrays
# # src_tensor = tf.convert_to_tensor(src)
g = dgl.graph((src, dst))

# edges_enc = pd.DataFrame()
nodes_enc = pd.DataFrame()
nodes_enc['id'] = nodes_cpy['id']
nodes_enc['id_enc'] = lb_make.fit_transform(nodes_cpy['id'])
nodes_enc['geohash'] = lb_make.fit_transform(nodes_cpy['geohash'])
nodes_enc['category'] = lb_make.fit_transform(nodes_cpy['filtered_cat'])
nodes_enc['stars'] = lb_make.fit_transform(nodes_cpy['stars'])
nodes_enc['lat'] = nodes_cpy['latitude']
nodes_enc['lon'] = nodes_cpy['longitude']


# edges_enc['type'] = lb_make.fit_transform(edges['t'])
geohash = torch.tensor(nodes_enc['geohash'].to_numpy()).long()
category = torch.tensor(nodes_enc['category'].to_numpy()).long()
lat = torch.tensor(nodes_cpy['latitude'].to_numpy()).long()
lon =  torch.tensor(nodes_cpy['longitude'].to_numpy()).long()
stars =  torch.tensor(nodes_cpy['stars'].to_numpy()).long()
tensor_list =[geohash,lat,lon,category]
stacked_tensor = torch.stack(tensor_list,dim=1)
g.ndata['feat'] = stacked_tensor
g.ndata['label'] = stars
# print(stacked_tensor)
# Add node attributes to graph
# g.ndata['geohash'] = geohash
# g.ndata['category'] = category
# g.ndata['stars'] = stars
# g.ndata['lat'] = lat
# g.ndata['lon'] = lon

edges_enc = pd.DataFrame()
edges_enc['type'] = lb_make.fit_transform(er['edgetype'])
edges_enc['weight'] = er['weight']
edgetype = torch.tensor(edges_enc['type'].to_numpy()).long()
distance = torch.tensor(er['weight'].to_numpy()).long()

# Add edge attributes
g.edata['edgetype'] = edgetype
g.edata['distance'] = distance
g = dgl.add_self_loop(g)
# g = dgl.to_bidirected(g,True)

print(g)

Graph(num_nodes=3389, num_edges=806248,
      ndata_schemes={'feat': Scheme(shape=(4,), dtype=torch.int64), 'label': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'edgetype': Scheme(shape=(), dtype=torch.int64), 'distance': Scheme(shape=(), dtype=torch.int64)})


In [75]:
# Contruct a two-layer GNN model
import dgl
import dgl.nn as dglnn
from dgl.data import DGLDataset
import dgl.function as dglfn
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

import random

def process_graph(graph):
    # If your dataset is a node classification dataset, you will need to assign
    # masks indicating whether a node belongs to training, validation, and test set.
    n_nodes = graph.num_nodes()
    n_train = int(n_nodes * 0.8)
    n_val = int(n_nodes * 0.1)
    train_mask = torch.zeros(n_nodes, dtype=torch.bool)
    val_mask = torch.zeros(n_nodes, dtype=torch.bool)
    test_mask = torch.zeros(n_nodes, dtype=torch.bool)
    pop = [*range(n_nodes)]
    train_idx = random.sample(pop,n_train)
    for train_i in train_idx:
        pop.remove(train_i)
    val_idx = random.sample(pop,n_val)
    for val_i in val_idx:
        pop.remove(val_i)
    test_idx = pop
    # train_mask[:n_train] = True
    # val_mask[n_train:n_train + n_val] = True
    # test_mask[n_train + n_val:] = True
    train_mask[train_idx] = True
    val_mask[val_idx] = True
    test_mask[test_idx] = True
    graph.ndata['train_mask'] = train_mask
    graph.ndata['val_mask'] = val_mask
    graph.ndata['test_mask'] = test_mask
    return graph


# class GCN(nn.Module):
#     def __init__(self, in_feats, h_feats, num_classes):
#         super(GCN, self).__init__()
#         self.conv1 = GraphConv(in_feats, h_feats)
#         self.conv2 = GraphConv(h_feats, num_classes)

#     def forward(self, g, in_feat):
#         h = self.conv1(g, in_feat)
#         h = F.relu(h)
#         h = self.conv2(g, h)
#         return h  
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = dglnn.GraphConv(in_feats, h_feats)
        self.conv2 = dglnn.GraphConv(in_feats, h_feats)
        self.conv3 = dglnn.GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
#         h = F.softmax(h)
        h = self.conv3(g, h)
        return h

class GATConv(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GATConv, self).__init__()
        self.conv1 = dglnn.GATConv(n_feats=in_feats, out_feats=h_feats, num_heads=2)
        self.conv2 = dglnn.GATConv(in_feats=h_feats,  out_feats=num_classes, num_heads=2)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h    

class SAGE(nn.Module):
    def __init__(self, in_feats, h_feats, out_feats):
        super().__init__()
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=h_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=h_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h
    
def evaluate(model, graph, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)
    
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
#     optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

In [76]:
## GCN

graph = process_graph(g)

node_features = graph.ndata['feat']
node_labels = graph.ndata['label']
train_mask = graph.ndata['train_mask']
valid_mask = graph.ndata['val_mask']
test_mask = graph.ndata['test_mask']
# n_features = node_features.shape[1]
graph.ndata['label'] = graph.ndata['label'].type(torch.LongTensor)

model = GCN(graph.ndata['feat'].shape[1], 16, len(set(nodes_enc['stars'])))
train(graph, model)
# print(graph.ndata['feat'].shape[1])

In epoch 0, loss: 53.360, val acc: 0.024 (best 0.024), test acc: 0.006 (best 0.006)
In epoch 5, loss: 21.426, val acc: 0.068 (best 0.068), test acc: 0.065 (best 0.065)
In epoch 10, loss: 10.211, val acc: 0.266 (best 0.266), test acc: 0.276 (best 0.276)
In epoch 15, loss: 1.935, val acc: 0.464 (best 0.464), test acc: 0.500 (best 0.500)
In epoch 20, loss: 5.013, val acc: 0.503 (best 0.503), test acc: 0.521 (best 0.521)
In epoch 25, loss: 3.040, val acc: 0.503 (best 0.503), test acc: 0.521 (best 0.521)
In epoch 30, loss: 2.877, val acc: 0.284 (best 0.503), test acc: 0.276 (best 0.521)
In epoch 35, loss: 1.882, val acc: 0.506 (best 0.506), test acc: 0.521 (best 0.521)
In epoch 40, loss: 1.742, val acc: 0.500 (best 0.506), test acc: 0.515 (best 0.521)
In epoch 45, loss: 1.657, val acc: 0.299 (best 0.506), test acc: 0.288 (best 0.521)
In epoch 50, loss: 1.602, val acc: 0.503 (best 0.506), test acc: 0.518 (best 0.521)
In epoch 55, loss: 1.480, val acc: 0.305 (best 0.506), test acc: 0.291 (bes